In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define Classes and Functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Convolutional LSTM Model for Quantile Regression with Multi-Step Prediction
class Conv_LSTM_Quantile(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, num_quantiles, num_steps_ahead, num_filters, kernel_size):
        super(Conv_LSTM_Quantile, self).__init__()
        self.hidden_size = hidden_sizes
        self.num_layers = num_layers
        self.num_quantiles = num_quantiles
        self.num_steps_ahead = num_steps_ahead

        # Convolutional layer
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        
        self.lstms = nn.ModuleList()
        self.lstms.append(nn.LSTM(num_filters, hidden_sizes[0], batch_first=True))

        for i in range(1, num_layers):
            self.lstms.append(nn.LSTM(hidden_sizes[i-1], hidden_sizes[i], batch_first=True))
        
        self.fc_layers = nn.ModuleList([nn.Linear(hidden_sizes[-1], num_quantiles) for _ in range(num_steps_ahead)])

    def forward(self, x):
        # Convolutional layer expects input of shape (batch_size, in_channels, seq_length)
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = x.permute(0, 2, 1)  # Convert back to (batch_size, seq_length, num_filters)
        
        h = x
        for lstm in self.lstms:
            out, _ = lstm(h)
            h = out
 
        # Compute the outputs for each step ahead
        step_outputs = [fc(out[:, -1, :]) for fc in self.fc_layers]
        
        # Stack the step outputs
        output = torch.stack(step_outputs, dim=1)  # [batch_size, num_steps_ahead, num_quantiles]
        return output

def quantile_loss(preds, target, quantiles):
    losses = []
    for i, quantile in enumerate(quantiles):
        errors = target[:, :, i] - preds[:, :, i]
        losses.append(torch.mean(torch.max((quantile - 1) * errors, quantile * errors)))
    return torch.mean(torch.stack(losses))

def split_data(data, input_size, output_size, train_ratio, seed):
    X, y = [], [] 
    total_size = input_size + output_size
    for i in range(len(data) - total_size + 1):
        X.append(features[i:i + input_size])                            
        y.append(target[i + input_size:i + total_size])        
    # 2. shuffle batches and split into train/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = train_ratio, random_state=seed)
    
    return X_train, X_test, Y_train, Y_test

def evaluate_model(model, dataloader, quantiles):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    
    with torch.no_grad():  # Disable gradient calculation
        for inputs, targets in dataloader:
            targets = targets.unsqueeze(-1).expand(-1, -1, len(quantiles))  # Reshape targets
            outputs = model(inputs)  # Forward pass
            all_preds.append(outputs)
            all_targets.append(targets)
    
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    return all_preds, all_targets

pm = "\u00B1"

Ethereum = pd.read_csv('data/coin_Ethereum.csv')
data = Ethereum.iloc[:, 4:]
features = data[['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']]
features = MinMaxScaler().fit_transform(features) # normalise input
target = data['Close']
target_reshaped = np.array(target).reshape(-1, 1) # normalise output
scaler = MinMaxScaler(feature_range=(0, 1))
target = scaler.fit_transform(target_reshaped).flatten()

# Define our parameters
input_size = 6        # 6 steps input
output_size = 5       # 5 steps output
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_size)]
mae_steps = [[] for _ in range(output_size)]
mape_steps = [[] for _ in range(output_size)]

## Quantile Version
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
num_quantiles = len(quantiles)  # Number of quantiles to predict
num_experiments = 30    # default: 30 (change to lower value for less intensive runs)

rmse, mae, mape = [], [], []
train_rmse, train_mae, train_mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
train_rmse_005, train_rmse_025, train_rmse_050, train_rmse_075, train_rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_size)]
train_rmse_steps = [[] for _ in range(output_size)]
mae_steps = [[] for _ in range(output_size)]
train_mae_steps = [[] for _ in range(output_size)]
mape_steps = [[] for _ in range(output_size)]
train_mape_steps = [[] for _ in range(output_size)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test = split_data(data, input_size, output_size, train_ratio, seed)
    train_dataset = Time_Series_Dataset(X_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False) # changing batch size affect model accuracy significantly
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False) 

    hidden_sizes = [64, 32]  # Hidden sizes for each LSTM layer
    num_layers = 2  # Number of LSTM layers
    num_filters = 64  # Number of filters for Conv1D layer
    kernel_size = 2  # Kernel size for Conv1D layer
    num_steps_ahead = 5
    
    # Create the model
    model = Conv_LSTM_Quantile(input_size, hidden_sizes, num_layers, num_quantiles, num_steps_ahead, num_filters, kernel_size)

    # Loss and optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_dataloader: # inputs = X_train, targets = y_train
            
            targets = targets.unsqueeze(-1).expand(-1, -1, len(quantiles)) # [size, num_steps_ahead, num_quantiles]
            
            optimizer.zero_grad()
            outputs = model(inputs)  # Forward pass - [size, num_steps_ahead, num_quantiles]
            loss = quantile_loss(outputs, targets, quantiles)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        
        # Print epoch information for the first epoch and every 10th epoch thereafter
        # if epoch == 1 or epoch % 10 == 0 or epoch == num_epochs:
            # print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.6f}')

    # Evaluate the model on the test set
    predicted_values, actual_values = evaluate_model(model, test_dataloader, quantiles)
    predicted_values = predicted_values.numpy()
    actuals = actual_values.numpy()[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]

    ## Calculate RMSE here for test set
    mse_005 = mean_squared_error(pred_005, actuals)
    mse_025 = mean_squared_error(pred_025, actuals)
    mse_050 = mean_squared_error(pred_050, actuals)
    mse_075 = mean_squared_error(pred_075, actuals)
    mse_095 = mean_squared_error(pred_095, actuals)

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals)

    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values, predicted_050))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_050))

    for step in range(output_size):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))

    # Evaluate the model on the training set
    train_predicted_values, train_actual_values = evaluate_model(model, train_dataloader, quantiles)
    train_predicted_values = train_predicted_values.numpy()
    train_actuals = train_actual_values.numpy()[:, :, 0]

    train_pred_005 = train_predicted_values[:, :, 0]
    train_pred_025 = train_predicted_values[:, :, 1]
    train_pred_050 = train_predicted_values[:, :, 2]
    train_pred_075 = train_predicted_values[:, :, 3]
    train_pred_095 = train_predicted_values[:, :, 4]

    ## Calculate RMSE here for training set
    train_mse_005 = mean_squared_error(train_pred_005, train_actuals)
    train_mse_025 = mean_squared_error(train_pred_025, train_actuals)
    train_mse_050 = mean_squared_error(train_pred_050, train_actuals)
    train_mse_075 = mean_squared_error(train_pred_075, train_actuals)
    train_mse_095 = mean_squared_error(train_pred_095, train_actuals)

    train_rmse_005.append(np.sqrt(train_mse_005))
    train_rmse_025.append(np.sqrt(train_mse_025))
    train_rmse_050.append(np.sqrt(train_mse_050))
    train_rmse_075.append(np.sqrt(train_mse_075))
    train_rmse_095.append(np.sqrt(train_mse_095))

    train_pred_values = train_predicted_values.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(train_predicted_values.shape)
    train_actual_values = scaler.inverse_transform(train_actuals)

    train_predicted_005 = train_pred_values[:, :, 0]
    train_predicted_025 = train_pred_values[:, :, 1]
    train_predicted_050 = train_pred_values[:, :, 2]
    train_predicted_075 = train_pred_values[:, :, 3]
    train_predicted_095 = train_pred_values[:, :, 4]

    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_050))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_050))

    for step in range(output_size):
        train_mse_step = mean_squared_error(train_pred_050[:, step], train_actuals[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))
        train_mae_steps[step].append(mean_absolute_error(train_predicted_050[:, step], train_actual_values[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_predicted_050[:, step], train_actual_values[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

print(f'Bitcoin Conv-LSTM Average RMSE across {output_size} time steps at different quantiles:')
print(f'Quantile 0.05: {np.mean(rmse_005):.6f} {pm} {np.std(rmse_005):.6f}')
print(f'Quantile 0.25: {np.mean(rmse_025):.6f} {pm} {np.std(rmse_025):.6f}')
print(f'Quantile 0.50: {np.mean(rmse_050):.6f} {pm} {np.std(rmse_050):.6f}')
print(f'Quantile 0.75: {np.mean(rmse_075):.6f} {pm} {np.std(rmse_075):.6f}')
print(f'Quantile 0.95: {np.mean(rmse_095):.6f} {pm} {np.std(rmse_095):.6f}')

print(f"Training RMSE across {output_size} time steps at different quantiles:")
print(f'Quantile 0.05: {np.mean(train_rmse_005):.6f} {pm} {np.std(train_rmse_005):.6f}')
print(f'Quantile 0.25: {np.mean(train_rmse_025):.6f} {pm} {np.std(train_rmse_025):.6f}')
print(f'Quantile 0.50: {np.mean(train_rmse_050):.6f} {pm} {np.std(train_rmse_050):.6f}')
print(f'Quantile 0.75: {np.mean(train_rmse_075):.6f} {pm} {np.std(train_rmse_075):.6f}')
print(f'Quantile 0.95: {np.mean(train_rmse_095):.6f} {pm} {np.std(train_rmse_095):.6f}')

print("--------------------------------------------")

print(f"Conv-LSTM Quantile Regression Average Performance: RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, MAE: {np.mean(mae):.4f} {pm} {np.std(mae):.4f}, MAPE: {np.mean(mape)*100:.4f}% {pm} {np.std(mape)*100:.4f}%")
for step in range(output_size):
    print(f"At time step {step + 1}, predictions have RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, MAE: {np.mean(mae_steps[step]):.4f} {pm} {np.std(mae_steps[step]):.4f} and MAPE: {np.mean(mape_steps[step])*100:.4f}% {pm} {np.std(mape_steps[step])*100:.4f}%")

print(f"Training Performance: RMSE: {np.mean(train_rmse_050):.4f} {pm} {np.std(train_rmse_050):.4f}, MAE: {np.mean(train_mae):.4f} {pm} {np.std(train_mae):.4f}, MAPE: {np.mean(train_mape)*100:.4f}% {pm} {np.std(train_mape)*100:.4f}%")
for step in range(output_size):
    print(f"At time step {step + 1}, training predictions have RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, MAE: {np.mean(train_mae_steps[step]):.4f} {pm} {np.std(train_mae_steps[step]):.4f} and MAPE: {np.mean(train_mape_steps[step])*100:.4f}% {pm} {np.std(train_mape_steps[step])*100:.4f}%")


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done
Bitcoin Conv-LSTM Average RMSE across 5 time steps at different quantiles:
Quantile 0.05: 0.039247 ± 0.004079
Quantile 0.25: 0.024107 ± 0.002549
Quantile 0.50: 0.021247 ± 0.003101
Quantile 0.75: 0.026720 ± 0.005714
Quantile 0.95: 0.040494 ± 0.006320
Training RMSE across 5 time steps at different quantiles:
Quantile 0.05: 0.040351 ± 0.001837
Quanti

In [2]:
quantile_dfs = []
for i in range(pred_values.shape[2]):
    quantile_df = pd.DataFrame(
        pred_values[:, :, i], 
        columns=[f'Quantile_{i+1}_Timestep_{j+1}' for j in range(pred_values.shape[1])]
    )
    quantile_dfs.append(quantile_df)
predicted_df = pd.concat(quantile_dfs, axis=1)

predicted_df.to_csv('ethereum_multi_clstm_quantile_results.csv')
pd.DataFrame(actual_values).to_csv('ethereum_multi_clstm_quantile_actual.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('ethereum_multi_clstm_quantile_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('ethereum_multi_clstm_quantile_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('ethereum_multi_clstm_quantile_mape.csv')